In [24]:
import sys
import os
from tqdm import tqdm
sys.path.append('/Users/luisgonzales/Thesis/code_base/deep_calibration/rbergomi')
print(os.getcwd())

/Users/luisgonzales/Thesis/code_base/deep_calibration/main


In [37]:
import rbergomi
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from utils import *
np.random.seed(42)

## Define Grid (like in Horvath et al. 2021)

In [31]:
strikes=np.array([0.5,0.6,0.7,0.8,0.9,1.0,1.1,1.2,1.3,1.4,1.5 ])
maturities=np.array([0.1,0.3,0.6,0.9,1.2,1.5,1.8,2.0 ])

## Generate Synthetic MC Data for 80.000 different parameter combinations 

In [28]:
n = 100
N = 60000
T = 2.0  # Longest maturity
method = 'rdonsker'
S0 = 1.0
r = 0.0  # Risk-free rate assumed 0

In [30]:
num_combinations = 80000
parameter_combinations = {
    "a": np.random.uniform(-0.49, -0.3, num_combinations),
    "rho": np.random.uniform(-0.99, -0.1, num_combinations),
    #"xi": np.random.uniform(0.01, 0.1, num_combinations),
    "xi": [np.random.uniform(0.01, 0.1, 8) for _ in range(num_combinations)]
    "eta": np.random.uniform(0.5, 3.0, num_combinations),
}

In [32]:
option_prices = np.zeros((num_combinations, len(maturities), len(strikes)))

for i in tqdm(range(num_combinations), desc="Simulating parameter combinations..."):
    a = parameter_combinations["a"][i]
    rho = parameter_combinations["rho"][i]
    xi = parameter_combinations["xi"][i]
    eta = parameter_combinations["eta"][i]

    # paths
    rB = rBergomi(n=n, N=N, T=T, a=a, method=method)
    dW1 = rB.dW1()
    dW2 = rB.dW2()
    Y = rB.Y(dW1)
    dB = rB.dB(dW1, dW2, rho=rho)
    V = rB.V(Y, xi=xi, eta=eta)
    S = rB.S(V, dB, S0=S0)

    # Loop over each maturity and strike, compute option prices
    for m_idx, T_i in enumerate(maturities):
        t_idx = int(T_i * n)  # Index for this maturity

        S_T = S[:, t_idx]
        for k_idx, K in enumerate(strikes):
            payoff = np.maximum(S_T - K, 0)
            option_prices[i, m_idx, k_idx] = np.mean(payoff)

# Save the pricing tensor for further IV inversion or NN training
np.save("option_prices_80000x8x11.npy", option_prices)

Simulating parameter combinations...:   0%|          | 0/80000 [00:01<?, ?it/s]


ValueError: operands could not be broadcast together with shapes (60000,101) (1,201) 

In [33]:
xi_curve, t_grid, xi_pieces = generate_piecewise_forward_variance(T=model.T, n=model.n)

plt.figure(figsize=(10, 6))
plt.plot(xi_curve, t_grid, label='Piecewise Forward Variance Curve')
plt.show()

NameError: name 'generate_piecewise_forward_variance' is not defined